Otwieranie i zamykanie pliku w Pythonie

In [1]:
file = open('kosowskie1.txt')
file

<_io.TextIOWrapper name='kosowskie1.txt' mode='r' encoding='cp1250'>

try-finally / with statements

In [2]:
reader = open('kosowskie1.txt')
try:
    print('processing data')
finally:
    reader.close()

with open('kosowskie1.txt') as reader:
    print('with processing data')

processing data
with processing data


Text File Types

In [3]:
with open('kosowskie1.txt') as file:
    print(type(file))
with open('kosowskie1.txt', 'r') as file:
    print(type(file))
with open('kosowskie1.txt', 'w') as file:
    print(type(file))
with open('kosowskie1.txt', 'rb') as file:
    print(type(file))
with open('kosowskie1.txt', 'wb') as file:
    print(type(file))

<class '_io.TextIOWrapper'>
<class '_io.TextIOWrapper'>
<class '_io.TextIOWrapper'>
<class '_io.BufferedReader'>
<class '_io.BufferedWriter'>


Raw File Types

In [4]:
file = open('kosowskie1.txt', 'rb', buffering=0)
type(file)

_io.FileIO

Reading and Writing Opened Files

In [6]:
with open('kosowskie1.txt') as reader:
    print(reader.read())

Znaki specjalne $#@!1312/
********
Plik do odczytu


In [7]:
with open('kosowskie1.txt', 'r') as reader:
    
    print(reader.readline(5))
    
    print(reader.readline(5))
    print(reader.readline(5))
    print(reader.readline(5))
    print(reader.readline(5))

Znaki
 spec
jalne
 $#@!
1312/


In [8]:
f = open('kosowskie1.txt')
f.readlines()

f = open('kosowskie1.txt')
list(f)

['Znaki specjalne $#@!1312/\n', '********\n', 'Plik do odczytu']

Iterating Over Each Line in the File

In [9]:
with open('kosowskie1.txt', 'r') as reader:
     line = reader.readline()
     while line != '':
         print(line, end='')
         line = reader.readline()

Znaki specjalne $#@!1312/
********
Plik do odczytu

In [10]:
with open('kosowskie1.txt', 'r') as reader:
     for line in reader.readlines():
         print(line, end='')

Znaki specjalne $#@!1312/
********
Plik do odczytu

In [11]:
with open('kosowskie1.txt', 'r') as reader:
     for line in reader:
         print(line, end='')

Znaki specjalne $#@!1312/
********
Plik do odczytu

In [12]:
with open('kosowskie1.txt', 'r') as reader:
    readOnly = reader.readlines()

with open('kosowskie1.txt', 'w') as writer:
    for word in reversed(readOnly):
        writer.write(word)

with open('kosowskie1.txt', 'w') as writer:
    writer.writelines(reversed(readOnly))

Working With Bytes

In [14]:
with open('bity21348.txt', 'rb') as reader:
     print(reader.readline())

b''


In [15]:
with open('A.png', 'rb') as byte_reader:
     print(byte_reader.read(1))
     print(byte_reader.read(3))
     print(byte_reader.read(2))
     print(byte_reader.read(1))
     print(byte_reader.read(1))

b'\x89'
b'PNG'
b'\r\n'
b'\x1a'
b'\n'


Appending to a File

In [16]:
with open('AppendingKosowskie.txt', 'a') as a_writer:
    a_writer.write('\nBeagle')
with open('AppendingKosowskie.txt', 'r') as reader:
     print(reader.read())

Appending file by Emanuel Kosowski 21348
Beagle
Beagle


Working With Two Files at the Same Time

In [17]:
d_path = 'AppendingKosowskie.txt'
d_r_path = 'AppendingKosowskie2.txt'
with open(d_path, 'r') as reader, open(d_r_path, 'w') as writer:
    file_breeds = reader.readlines()
    writer.writelines(reversed(file_breeds))

Creating Your Own Context Manager

In [18]:
class my_file_reader():
    def __init__(self, file_path):
        self.__path = file_path
        self.__file_object = None

    def __enter__(self):
        self.__file_object = open(self.__path)
        return self

    def __exit__(self, type, val, tb):
        self.__file_object.close()
        
with my_file_reader('kosowskie1.txt') as reader:
    pass

In [20]:
class PngReader():
    # Every .png file contains this in the header.  Use it to verify
    # the file is indeed a .png.
    _expected_magic = b'\x89PNG\r\n\x1a\n'

    def __init__(self, file_path):
        # Ensure the file has the right extension
        if not file_path.endswith('.png'):
            raise NameError("File must be a '.png' extension")
        self.__path = file_path
        self.__file_object = None

    def __enter__(self):
        self.__file_object = open(self.__path, 'rb')

        magic = self.__file_object.read(8)
        if magic != self._expected_magic:
            raise TypeError("The File is not a properly formatted .png file!")

        return self

    def __exit__(self, type, val, tb):
        self.__file_object.close()

    def __iter__(self):
        # This and __next__() are used to create a custom iterator
        # See https://dbader.org/blog/python-iterators
        return self

    def __next__(self):
        # Read the file in "Chunks"
        # See https://en.wikipedia.org/wiki/Portable_Network_Graphics#%22Chunks%22_within_the_file

        initial_data = self.__file_object.read(4)

        # The file hasn't been opened or reached EOF.  This means we
        # can't go any further so stop the iteration by raising the
        # StopIteration.
        if self.__file_object is None or initial_data == b'':
            raise StopIteration
        else:
            # Each chunk has a len, type, data (based on len) and crc
            # Grab these values and return them as a tuple
            chunk_len = int.from_bytes(initial_data, byteorder='big')
            chunk_type = self.__file_object.read(4)
            chunk_data = self.__file_object.read(chunk_len)
            chunk_crc = self.__file_object.read(4)
            return chunk_len, chunk_type, chunk_data, chunk_crc

with PngReader('A.png') as reader:
    for l, t, d, c in reader:
        print(f"{l:05}, {t}, {c}")

00013, b'IHDR', b'\xeb!\xb3\xcf'
00006, b'bKGD', b'\xa0\xbd\xa7\x93'
08192, b'IDAT', b'a\x80\x92\xa6'
08192, b'IDAT', b'\xf0\xb6\xf2\xf4'
08192, b'IDAT', b'\x97\xef\x96\x9c'
08192, b'IDAT', b'h\xdc\xb0\xe0'
08192, b'IDAT', b'@\xf6\xb1~'
08192, b'IDAT', b'\x96PVj'
03384, b'IDAT', b'\x1d\x1e%8'
00000, b'IEND', b'\xaeB`\x82'
